In [269]:
%load_ext autoreload
%autoreload 2
import spike_queries
import pandas as pd
import tqdm
import pickle
import random
import itertools

import torch
from transformers import BertTokenizer
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import BertForSequenceClassification, AdamW, BertConfig, BertModel, AutoTokenizer, AutoModel, PreTrainedTokenizerFast
import numpy as np
from typing import List
from torch import nn
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from typing import Dict, Tuple

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [167]:
with open("queries.txt", "r") as f:
    queries = f.readlines()
    queries = [l.strip() for l in queries]

In [168]:
queries2results = dict()

In [169]:
dataset_name = "covid19"
num_results = 100
query_type = "syntactic"

for q in tqdm.tqdm(queries, total = len(queries)):
    df = spike_queries.perform_query(q, dataset_name, num_results, query_type) #previously: word=Hawaii
    queries2results[q] = df

100%|██████████| 67/67 [05:45<00:00,  5.15s/it]


In [170]:
with open("queries2results.pickle", "wb") as f:
    pickle.dump(queries2results, f)

In [171]:
with open("queries2results.pickle", "rb") as f:
    queries2results = pickle.load(f)

In [176]:
data = []
for q in queries:
    #print(q)
    results = queries2results[q]
    sents, arg1_first, arg2_first = results["sentence_text"].tolist(), results["arg1_first_index"].tolist(), results["arg2_first_index"].tolist()
    arg1_last, arg2_last = results["arg1_last_index"].tolist(), results["arg2_last_index"].tolist()
    
    sents_with_args = []
    for s,arg1_ind,arg2_ind, arg1_ind_last, arg2_ind_last in zip(sents,arg1_first,arg2_first, arg1_last, arg2_last):
        s_lst = s.split(" ")
        if arg1_ind > arg2_ind:
            arg1_ind, arg2_ind = arg2_ind, arg1_ind
            arg1_ind_last, arg2_ind_last = arg2_ind_last, arg1_ind_last
            arg1_str, arg2_str = "ARG2:", "ARG1:"
        else:
            arg1_str, arg2_str = "ARG1:", "ARG2:"
        s_with_args = s_lst[:arg1_ind] + [arg1_str+s_lst[arg1_ind]] + s_lst[arg1_ind+1:arg2_ind] + [arg2_str+s_lst[arg2_ind]] + s_lst[arg2_ind+1:]
        s_with_args = " ".join(s_with_args)
        sents_with_args.append({"sent": s_with_args, "start_1": arg1_ind, "start_2": arg2_ind, "end_1": arg1_ind_last,
                               "end_2": arg2_ind_last})
    
    max_number_of_pairs = 100
    pairs = list(itertools.product(sents_with_args, repeat=2))
    print(len(pairs))
    random.shuffle(pairs)
    for pair in pairs[:max_number_of_pairs]:
        data.append({"first": pair[0]["sent"], "second": pair[1]["sent"], "query": q, "first_arg1": (pair[0]["start_1"], pair[0]["end_1"]),
                    "first_arg2": (pair[0]["start_2"], pair[0]["end_2"]), "second_arg1": (pair[1]["start_1"], pair[1]["end_1"]),
                   "second_arg2": (pair[1]["start_2"], pair[1]["end_2"])})

10000
1225
10000
10000
36
10000
10000
10000
81
10000
10000
10000
10000
10000
2601
10000
5041
10000
10000
576
10000
4096
10000
10000
169
6241
10000
4
10000
841
3721
10000
10000
441
2704
169
10000
3969
361
10000
10000
10000
576
10000
16
25
196
10000
10000
576
169
3600
529
3844
10000
10000
10000
576
576
10000
121
25
529
289
10000
2916
10000


In [177]:
random.shuffle(data)
data[0]

{'first': 'ARG1:Bronchiolitis obliterans ( BO ) is a severe complication after allogeneic hematopoietic stem ARG2:cell transplantation with an unfavorable prognosis .',
 'second': 'ARG1:Thrombocytopenia is a common complication of influenza ARG2:virus infection , and its severity predicts the clinical outcome of critically ill patients .',
 'query': 'arg1:[e]stroke is a $complication of arg2:flu :vaccination',
 'first_arg1': (0, 0),
 'first_arg2': (13, 13),
 'second_arg1': (0, 0),
 'second_arg2': (7, 7)}

In [178]:
import json

with open("data.txt", "w") as f:
    for d in data:
        first, second = d["first"], d["second"]
        first_arg1 = d["first_arg1"]
        first_arg2 = d["first_arg2"]
        second_arg1 = d["second_arg1"]
        second_arg2 = d["second_arg2"]
        
        elems = [first, second, first_arg1, first_arg2, second_arg1, second_arg2]
        keys = ["first", "second", "first_arg1", "first_arg2"]
        
        f.write(json.dumps(d) + "\n")
        #f.write(d["first"] + "\t" + d["second"] + "\t" + d["query"] + "\t" + "-".join(d["first_arg1"])+ "\n")

In [179]:
random.shuffle(data)
data[9]

{'first': 'The equine ARG1:influenza virus vaccine with CNPV vector expresses the hemagglutinin genes of the H3N8 Newmarket and Kentucky ARG2:strains and contains a polymer adjuvant ( Carbopol ; Merial Ltd. ) .',
 'second': 'The reassortant H5N1 virus A/Harbin/Re-1/2003 ( ARG1:Re-1 ) derives its ARG2:HA and NA genes from A/goose/Guangdong/1/96(H5N1 ) virus , and six internal genes from the high-growth A/Puerto Rico/8/34 ( PR8 ) virus .',
 'query': 'arg1:something $virus affects arg2:something $genes',
 'first_arg1': (2, 2),
 'first_arg2': (18, 18),
 'second_arg1': (6, 6),
 'second_arg2': (10, 10)}

In [137]:
queries2results[queries[0]]

,sentence_id,arg1,arg2,title,article_link,pmc_id,doi,pdf_sha,pubmed_id,msft_id,arg1_first_index,arg1_last_index,arg2_first_index,arg2_last_index,sentence_text,paragraph_text
0,8903,Mouse hepatitis virus infection of mice,long-term depletion of lactate dehydrogenase-e...,Mouse hepatitis virus infection of mice causes...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,PMC7173247,10.1016/0168-1702(95)00092-5,NaN,8837897.0,NaN,0,5,7,13,Mouse hepatitis virus infection of mice causes...,Mouse hepatitis virus infection of mice causes...
1,8903,Mouse hepatitis virus infection of mice,lactate dehydrogenase-elevating virus-permissi...,Mouse hepatitis virus infection of mice causes...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,PMC7173247,10.1016/0168-1702(95)00092-5,NaN,8837897.0,NaN,0,5,10,13,Mouse hepatitis virus infection of mice causes...,Mouse hepatitis virus infection of mice causes...
2,8903,Mouse hepatitis virus infection of mice,T lymphocyte alterations,Mouse hepatitis virus infection of mice causes...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,PMC7173247,10.1016/0168-1702(95)00092-5,NaN,8837897.0,NaN,0,5,15,17,Mouse hepatitis virus infection of mice causes...,Mouse hepatitis virus infection of mice causes...
3,8903,mice,long-term depletion of lactate dehydrogenase-e...,Mouse hepatitis virus infection of mice causes...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,PMC7173247,10.1016/0168-1702(95)00092-5,NaN,8837897.0,NaN,5,5,7,13,Mouse hepatitis virus infection of mice causes...,Mouse hepatitis virus infection of mice causes...
4,8903,mice,lactate dehydrogenase-elevating virus-permissi...,Mouse hepatitis virus infection of mice causes...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,PMC7173247,10.1016/0168-1702(95)00092-5,NaN,8837897.0,NaN,5,5,10,13,Mouse hepatitis virus infection of mice causes...,Mouse hepatitis virus infection of mice causes...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,851682,Novel 2019 coronavirus ( COVID-19 ) infection,a respiratory disease,Acute lower limb ischemia as clinical presenta...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,PMC7417264,10.1016/j.avsg.2020.08.004,NaN,32791191.0,NaN,0,6,9,11,Novel 2019 coronavirus ( COVID-19 ) infection ...,Novel 2019 coronavirus (COVID-19) infection us...
96,851682,COVID-19,a respiratory disease,Acute lower limb ischemia as clinical presenta...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,PMC7417264,10.1016/j.avsg.2020.08.004,NaN,32791191.0,NaN,4,4,9,11,Novel 2019 coronavirus ( COVID-19 ) infection ...,Novel 2019 coronavirus (COVID-19) infection us...
97,855160,Novel 2019 coronavirus infection,a respiratory disease,Acute lower limb ischemia as clinical presenta...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,PMC7417264,10.1016/j.avsg.2020.08.004,NaN,32791191.0,NaN,0,3,6,8,Novel 2019 coronavirus infection usually cause...,Novel 2019 coronavirus infection usually cause...
98,927797,ADV infection,symptoms of low respiratory illness,Epidemiology of Acute Respiratory Infections i...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4...,PMC4010508,10.1371/journal.pone.0096674,NaN,24797911.0,NaN,0,1,4,8,ADV infection usually causes symptoms of low r...,ADV infection usually causes symptoms of low r...


In [181]:
tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')

2020-10-19 19:00:04.273 INFO    filelock: Lock 139840848406224 acquired on /home/shauli/.cache/torch/transformers/199e28e62d2210c23d63625bd9eecc20cf72a156b29e2a540d4933af4f50bda1.4b6b9f5d813f7395e7ea533039e02deb1723d8fd9d8ba655391a01a69ad6223d.lock


2020-10-19 19:00:05.011 INFO    filelock: Lock 139840848406224 released on /home/shauli/.cache/torch/transformers/199e28e62d2210c23d63625bd9eecc20cf72a156b29e2a540d4933af4f50bda1.4b6b9f5d813f7395e7ea533039e02deb1723d8fd9d8ba655391a01a69ad6223d.lock


2020-10-19 19:00:05.757 INFO    filelock: Lock 139839988439312 acquired on /home/shauli/.cache/torch/transformers/e3debd8fbdf40874753724814ee0520f612b577b26c8755bca485103b47cd3bc.60287becc5ab96d85a4bf377eb90feaf3b9c80d3b23e84311dccd3588f56d4fb.lock


2020-10-19 19:00:07.023 INFO    filelock: Lock 139839988439312 released on /home/shauli/.cache/torch/transformers/e3debd8fbdf40874753724814ee0520f612b577b26c8755bca485103b47cd3bc.60287becc5ab96d85a4bf377eb90feaf3b9c80d3b23e84311dccd3588f56d4fb.lock


In [390]:
class BertModel(pl.LightningModule):

    def __init__(self, train_dataset: Dataset, dev_dataset: Dataset, batch_size, device: str, mode: str = "eval"):
        
        super().__init__()
        
        self.device = device
        config = AutoConfig.from_pretrained('allenai/scibert_scivocab_uncased', output_hidden_states=True)
        self.tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
        self.model = AutoModel.from_pretrained('allenai/scibert_scivocab_uncased', config=config)    
        self.train_dataset = train_dataset
        self.dev_dataset = dev_dataset
        
        if mode == "eval":
            
            self.model.eval()
        else:
            self.model.train()
            
        self.train_gen = torch.utils.data.DataLoader(self.train_dataset, batch_size=batch_size, drop_last=False, shuffle=True)
        self.dev_gen = torch.utils.data.DataLoader(self.dev_dataset, batch_size=batch_size, drop_last=False, shuffle=True)
        self.acc = None

        
    def tokenize(self, original_sentence: List[str]) -> Tuple[List[str], Dict[int, int]]:

        """
        Parameters
        ----------
        Returns
        -------
        bert_tokens: The sentence, tokenized by BERT tokenizer.
        orig_to_tok_map: An output dictionary consisting of a mapping (alignment) between indices in the original tokenized sentence, and indices in the sentence tokenized by the BERT tokenizer. See https://github.com/google-research/bert
        """

        bert_tokens = ["[CLS]"]
        orig_to_tok_map = {}
        tok_to_orig_map = {}
        has_subwords = False
        is_subword = []

        for i, w in enumerate(original_sentence):
            tokenized_w = self.tokenizer.tokenize(w)
            has_subwords = len(tokenized_w) > 1
            is_subword.append(has_subwords)
            bert_tokens.extend(tokenized_w)

            orig_to_tok_map[i] = len(bert_tokens) - 1

        tok_to_orig_map = {}

        keys = list(sorted(orig_to_tok_map.keys()))
        for k, k2 in zip(keys, keys[1:]):
            for k3 in range(orig_to_tok_map[k], orig_to_tok_map[k2]):
                tok_to_orig_map[k3] = k

        bert_tokens.append("[SEP]")
        
        indexed_tokens = self.tokenizer.convert_tokens_to_ids(bert_tokens)
        tokens_tensor = torch.tensor([indexed_tokens])

        return (bert_tokens, orig_to_tok_map, tok_to_orig_map, tokens_tensor)
    
    
    def forward(self, x):
        
        outputs = self.model(x)
        states = outputs[0][0] #[seq_len, 768]
        return states
    
    def forward_with_loss_calculation(self, x, sent1_idx, sent2_idx, orig_to_tok_map, l):
        
        outputs = self.model(x)
        states = outputs[0][0] #[seq_len, 768]
        states = states/torch.norm(states, dim = 1, keepdim = True)
        sent1_arg1_vec, sent1_arg2_vec = states[orig_to_tok_map[sent1_idx[0]]], states[orig_to_tok_map[sent1_idx[1]]]  
        states_sent2 = states[orig_to_tok_map[l]:]
        
        
        sims_arg1 = sent1_arg1_vec @ states.T
        sims_arg2 = sent1_arg2_vec @ states.T
        sim_arg1_gold = sims_arg1[orig_to_tok_map[sent2_idx[0]]]
        sim_arg2_gold = sims_arg2[orig_to_tok_map[sent2_idx[1]]]
        
        idx_arg1 = torch.argsort(sims_arg1)
        idx_arg2 = torch.argsort(sims_arg2)
        
        most_sim_arg1, most_sim_arg2 = sims_arg1[idx_arg1[-2]], sims_arg2[idx_arg2[-2]]
        
        alpha = 0.1
        loss_arg1 = torch.max(torch.zeros(1), -sim_arg1_gold - most_sim_arg1 - alpha)
        loss_arg2 = torch.max(torch.zeros(1), -sim_arg2_gold - most_sim_arg2 - alpha)
        loss = loss_arg1 + loss_arg2
        
        return loss
    
    def training_step(self, batch, batch_nb):
        
        sents_concat, (sent1_arg1, sent1_arg2), (sent2_arg1, sent2_arg2), l = batch[0]    
        bert_tokens, orig_to_tok_map, tok_to_orig_map, tokens_tensor = self.tokenize(sents_concat.split(" "))        
        loss, logits = self.forward_with_loss_calculation(x, y)

        correct = logits.argmax(axis=1).int() == y.int()
        acc = torch.sum(correct).float() / len(y)

        return {'loss': loss, 'val_acc': acc}
    
    def validation_step(self, batch, batch_nb):
        sents_concat, (sent1_arg1, sent1_arg2), (sent2_arg1, sent2_arg2), l = batch[0]
        bert_tokens, orig_to_tok_map, tok_to_orig_map, tokens_tensor = self.tokenize(sents_concat.split(" "))        

        x = self.tokenize_and_pad(sents)
        loss, logits = self.forward_with_loss_calculation(x,y)

        correct = logits.argmax(axis=1).int() == y.int()
        acc = torch.sum(correct).float() / len(y)

        return {'val_loss': loss, 'val_acc': acc}
    
    def validation_end(self, outputs):
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        avg_acc = torch.stack([x['val_acc'] for x in outputs]).mean()
        print("Loss is {}".format(avg_loss))
        print("Accuracy is {}".format(avg_acc))
        self.acc = avg_acc
        return {'avg_val_loss': avg_loss}
    
    def configure_optimizers(self):
        # return torch.optim.SGD(self.parameters(), lr=0.005, momentum=0.9)
        return torch.optim.Adam(self.parameters(), weight_decay=1e-4)
    
    @pl.data_loader
    def train_dataloader(self):
        return self.train_gen

    @pl.data_loader
    def val_dataloader(self):
        # OPTIONAL
        # can also return a list of val dataloaders
        return self.dev_gen

In [391]:
class Dataset(torch.utils.data.Dataset):
    """Simple torch dataset class"""

    def __init__(self, data: List[Dict], device = "cpu"):

        self.data = data
        self.device = device

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        with torch.no_grad():
            
            d = self.data[index]
            sent1, sent2 = d["first"], d["second"]
            sent1_arg1, sent1_arg2 = d["first_arg1"][0], d["first_arg2"][0]
            sent2_arg1, sent2_arg2 = d["second_arg1"][0], d["second_arg2"][0]
            
            l = len(sent1.split(" ")) + 1 
            sent2_arg1, sent2_arg2 = sent2_arg1 + l, sent2_arg2 + l

            sent2 = sent2.replace("ARG1:", "").replace("ARG2:", "")
            sents_concat = sent1 + " ***** " + sent2 #sents_concat.split(" ")[l] is the first token in the 2nd sent
            
            return sents_concat, (sent1_arg1, sent1_arg2), (sent2_arg1, sent2_arg2), l

In [392]:
l = int(0.8 * len(data))
train_dataset, dev_dataset = Dataset(data[:l], "cpu"), Dataset(data[l:])

In [399]:
sents_concat, (sent1_arg1, sent1_arg2), (sent2_arg1, sent2_arg2), l = train_dataset[5]

In [400]:
sents_concat

'However , most of the infections remain subclinical , causing a latent ARG1:infection ( latent TB ) that can progress to active ARG2:pulmonary TB ( 10 % of latent TB cases ) ( Fatima et al. 2017 ) . ***** Presenting features of cough and fever subacutely progress to respiratory distress and acute respiratory distress sydrome ( ARDS ) in 8 - 19 % of patients , with the elderly and those with underlying comorbidities especially cardiovascular disease , diabetes mellitus , chronic pulmonary disorders or renal disease especially at risk https://www.epicentro.iss.it/coronavirus/bollettino/Report-COVID-2019_24_marzo_eng.pdf ( 42 ) ( 43 ) ( 44 ) ( 45 ) .'

In [401]:
lst = sents_concat.split(" ")
lst[sent2_arg2]

'acute'

In [396]:
model = BertModel(train_dataset, dev_dataset, 1, "cpu", "eval")

In [402]:
bert_tokens, orig_to_tok_map, tok_to_orig_map, tokens_tensor = model.tokenize(sents_concat.split(" "))
indexed_tokens = model.tokenizer.convert_tokens_to_ids(bert_tokens)
tokens_tensor = torch.tensor([indexed_tokens])

In [403]:
model.forward_with_loss_calculation(tokens_tensor, (sent1_arg1, sent1_arg2), (sent2_arg1, sent2_arg2), orig_to_tok_map, l)

tensor([0.], grad_fn=<AddBackward0>)

In [353]:
orig_to_tok_map

{0: 1,
 1: 2,
 2: 6,
 3: 7,
 4: 8,
 5: 9,
 6: 13,
 7: 15,
 8: 16,
 9: 17,
 10: 18,
 11: 19,
 12: 20,
 13: 22,
 14: 23,
 15: 25,
 16: 26,
 17: 28,
 18: 29,
 19: 31,
 20: 32,
 21: 34,
 22: 35,
 23: 36,
 24: 41,
 25: 42,
 26: 43,
 27: 45,
 28: 47,
 29: 48,
 30: 52,
 31: 53,
 32: 54,
 33: 55,
 34: 56,
 35: 57,
 36: 58,
 37: 59,
 38: 60,
 39: 62,
 40: 63,
 41: 64,
 42: 66,
 43: 67,
 44: 69,
 45: 70,
 46: 71,
 47: 72,
 48: 73,
 49: 74,
 50: 75}

In [379]:
torch.max

<function _VariableFunctionsClass.max>

In [389]:
torch.zeros(1)

tensor([0.])